# Data processing with Pandas, part 2

This week we will continue developing our skills using [pandas](https://pandas.pydata.org/) to process real data. 
The goal of this lesson is to use our data manipulation and analysis skills to detect weather anomalies (stormy winds) in Helsinki, during August 2017.

We will cover a number of useful techniques in pandas including:

- renaming columns
- iterating data frame rows and applying functions
- data aggregation
- repeating the analysis task 


## Input data
In the lesson this week we are using weather observation data from Helsinki, Finland from August 2017 when a severe thunderstorm "Kiira" hit souther Finland. [Kiira caused major damage in different pars of Helsinki](https://yle.fi/uutiset/osasto/news/saturday_night_storm_downs_trees_cuts_electricity_in_the_south/9773250) (*Source:* [YLE](https://yle.fi/uutiset/osasto/news/saturday_night_storm_downs_trees_cuts_electricity_in_the_south/9773250)).



![Markku Sipi/ Yle](img/Kiira-storm.PNG)

*Photo: Markku Sipi/YLE*


Notice that this time, we will read the **original data file** obtained from NOAA without any modifications to the data file (week 5 input data had been modified after downloading it from NOAA). The input data for this week is separated with varying amount of spaces (fixed width). The first lines and columns of the data looks like following:

```
      USAF  WBAN YR--MODAHRMN DIR SPD GUS CLG SKC L M H  VSB MW MW MW MW AW AW AW AW W TEMP DEWP    SLP  ...
    029740 99999 201708040000 114   6 *** *** BKN * * * 25.0 03 ** ** ** ** ** ** ** 2   58   56 1005.6  ...
    029740 99999 201708040020 100   6 ***  75 *** * * *  6.2 ** ** ** ** ** ** ** ** *   59   57 ******  ... 
    029740 99999 201708040050 100   5 ***  60 *** * * *  6.2 ** ** ** ** ** ** ** ** *   59   57 ******  ...
    029740 99999 201708040100 123   8 ***  63 OVC * * * 10.0 ** ** ** ** 23 ** ** ** *   59   58 1004.7  ...
    
```

**We will first work with a subset of the data (data for one single day):** [data/6591337447542dat_sample.txt](data/6591337447542dat_sample.txt)**, before repeating the process on a bigger data set (data for the whole month of August):**  [data/6591337447542dat_August.txt](data/6591337447542dat_August.txt).

## Reading the data



Because the data is **separated with varying amount of spaces**, we need to tell Pandas how to read it. We can control the delimiter with `sep` parameter following the documentation of the function [read_csv()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html):

![read_csv sep](img/read-csv-varying-spaces.png)

Based on this information, we should be able to separate the columns by varying number spaces of spaces with `sep='\s+'` -parameter.


Our data also included **No Data values** with varying number of `*` -characters which we also need to take into account when reading the data. 
We can tell pandas to consider those characters as NaNs by specifying `na_values=['*', '**', '***', '****', '*****', '******']`.

- Let's start by importing pandas and reading the data:

In [1]:
import pandas as pd
fp = "data/6591337447542dat_sample.txt"

# Read data using varying amount of spaces as separator and specifying * characters as NoData values
data = pd.read_csv(fp, sep='\s+', na_values=['*', '**', '***', '****', '*****', '******'])

- Let's see how the data looks by printing the first five rows with `head()` function

In [2]:
print(data.head())

    USAF   WBAN  YR--MODAHRMN  DIR  SPD  GUS   CLG  SKC   L   M ...      SLP  \
0  29740  99999  201708040000  114    6  NaN   NaN  BKN NaN NaN ...   1005.6   
1  29740  99999  201708040020  100    6  NaN  75.0  NaN NaN NaN ...      NaN   
2  29740  99999  201708040050  100    5  NaN  60.0  NaN NaN NaN ...      NaN   
3  29740  99999  201708040100  123    8  NaN  63.0  OVC NaN NaN ...   1004.7   
4  29740  99999  201708040120  110    7  NaN  70.0  NaN NaN NaN ...      NaN   

     ALT    STP  MAX  MIN  PCP01  PCP06  PCP24  PCPXX   SD  
0    NaN  999.2  NaN  NaN    NaN    NaN    NaN    NaN  0.0  
1  29.68    NaN  NaN  NaN    NaN    NaN    NaN    NaN  NaN  
2  29.65    NaN  NaN  NaN    NaN    NaN    NaN    NaN  NaN  
3    NaN  998.4  NaN  NaN    NaN    NaN    NaN    NaN  0.0  
4  29.65    NaN  NaN  NaN    NaN    NaN    NaN    NaN  NaN  

[5 rows x 33 columns]


## Renaming columns

- Let's continue and check what columns do we have.

Okay so we can see that the data was successfully read to the DataFrame and we also seemed to be able to convert the asterix (\*) characters into `NaN` -values. 

In [3]:
data.columns

Index(['USAF', 'WBAN', 'YR--MODAHRMN', 'DIR', 'SPD', 'GUS', 'CLG', 'SKC', 'L',
       'M', 'H', 'VSB', 'MW', 'MW.1', 'MW.2', 'MW.3', 'AW', 'AW.1', 'AW.2',
       'AW.3', 'W', 'TEMP', 'DEWP', 'SLP', 'ALT', 'STP', 'MAX', 'MIN', 'PCP01',
       'PCP06', 'PCP24', 'PCPXX', 'SD'],
      dtype='object')

As we can see, there are quite many columns, however, we won't be needing all of them in our thunderstorm analysis.

- Let's select only columns that we might need for detecting unexceptional weather conditions: `'YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN'`


In [4]:
# Specify a list of columns that will be selected from the DataFrame
select_cols = ['YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN']

# Do the selection
data = data[select_cols]

- Let's see what our data looks like now by printing **last** 5 rows and the datatypes.

In [5]:
# Show last five rows
print(data.tail())

# Check the data types
print("\nData-types:\n")
print(data.dtypes)

    YR--MODAHRMN  DIR  SPD   GUS  TEMP  MAX  MIN
67  201708042220  180   11   NaN    61  NaN  NaN
68  201708042250  190    8   NaN    59  NaN  NaN
69  201708042300  200    9  11.0    60  NaN  NaN
70  201708042320  190    8   NaN    59  NaN  NaN
71  201708042350  190    8   NaN    59  NaN  NaN

Data-types:

YR--MODAHRMN      int64
DIR               int64
SPD               int64
GUS             float64
TEMP              int64
MAX             float64
MIN             float64
dtype: object


The column names that we have are somewhat ackward. Let's change them into more intuitive ones. 
This can be done easily using the `rename()` -method and a dictionary that lists old and new column names.

<div class="alert alert-info">

**dictionaries**

[Dictionary](https://docs.python.org/2/tutorial/datastructures.html#dictionaries) is a spesific data structure in Python for storing key-value pairs. During this course, we will use dictionaries mainly when renaming columns in a pandas series, but dictionaries are useful for many different purposes! For more information about Python dictionaries, check out [this tutorial](https://realpython.com/python-dicts/).
    
</div>



We can define the new column names using a [dictionary](https://www.tutorialspoint.com/python/python_dictionary.htm) where we determine "`key: value`" -pairs, in which the original column name (the one which will be replaced) is the key, and the new column name is the value.

- Let's change:
   
   - `YR--MODAHRMN` column into `TIME`, 
   - `SPD` into `SPEED`, and
   - `GUS` into `GUST`

In [6]:
# Create the dictionary with old and new names
name_conversion_dict = {'YR--MODAHRMN': 'TIME', 'SPD': 'SPEED', 'GUS': 'GUST'}

# Let's see what they look like and what is the type
print(name_conversion_dict)
print(type(name_conversion_dict))

{'GUS': 'GUST', 'YR--MODAHRMN': 'TIME', 'SPD': 'SPEED'}
<class 'dict'>


From above we can see that we have successfully created a dictionary that is of type `dict`. 

- Now we can change the column names by passing that dictionary into parameter `columns` in `rename()` -function.

In [7]:
# Rename the columns
data = data.rename(columns=name_conversion_dict)

# Print the new columns
print(data.columns)

Index(['TIME', 'DIR', 'SPEED', 'GUST', 'TEMP', 'MAX', 'MIN'], dtype='object')


Perfect, now our column names are more easy to understand and use. 

## Reminder: basic calculations

Let's convert the wind speeds into meters per second values (m/s) as they are more familiar to us in Finland. This can be done with a formula **`m/s = mph x 0.44704`**:

In [8]:
# Convert speeds from miles to meters
data['SPEED'] = data['SPEED']*0.44704
data['GUST'] = data['GUST']*0.44704

In [9]:
data.head(10)

,TIME,DIR,SPEED,GUST,TEMP,MAX,MIN
0,201708040000,114,2.68224,NaN,58,NaN,NaN
1,201708040020,100,2.68224,NaN,59,NaN,NaN
2,201708040050,100,2.23520,NaN,59,NaN,NaN
3,201708040100,123,3.57632,NaN,59,NaN,NaN
4,201708040120,110,3.12928,NaN,59,NaN,NaN
5,201708040150,100,2.68224,NaN,61,NaN,NaN
6,201708040200,138,4.47040,5.81152,59,NaN,NaN
7,201708040220,120,4.47040,NaN,59,NaN,NaN
8,201708040250,100,4.02336,NaN,59,NaN,NaN
9,201708040300,108,4.02336,5.36448,59,NaN,NaN


- Let's check some basic statistics to understand our data better.

In [10]:
print(data.describe())

               TIME         DIR      SPEED       GUST       TEMP        MAX  \
count  7.200000e+01   72.000000  72.000000  20.000000  72.000000   2.000000   
mean   2.017080e+11  229.555556   5.153378   7.912608  61.513889  66.500000   
std    6.973834e+02  215.759248   1.679342   2.266045   3.175580   3.535534   
min    2.017080e+11   80.000000   2.235200   4.917440  58.000000  64.000000   
25%    2.017080e+11  117.750000   4.023360   5.811520  59.000000  65.250000   
50%    2.017080e+11  200.000000   4.917440   7.152640  61.000000  66.500000   
75%    2.017080e+11  220.000000   6.705600   9.946640  64.000000  67.750000   
max    2.017080e+11  990.000000   8.940800  12.964160  69.000000  69.000000   

             MIN  
count   2.000000  
mean   57.000000  
std     1.414214  
min    56.000000  
25%    56.500000  
50%    57.000000  
75%    57.500000  
max    58.000000  


Okey so from here we can see that there are varying number of observations per column (see the `count` -information). 

For example `SPD` and `TEMP` columns have 72 observations whereas `GUST` has only 20 observations and `MAX` and `MIN` have only 2 observations. From here we can already guess that `MAX` and `MIN` attributes are most probably not going to be useful for us. However, `GUST` might be.

- Let's explore further our data by checking the first 30 rows of it.



In [11]:
print(data.head(30))

            TIME  DIR    SPEED      GUST  TEMP   MAX   MIN
0   201708040000  114  2.68224       NaN    58   NaN   NaN
1   201708040020  100  2.68224       NaN    59   NaN   NaN
2   201708040050  100  2.23520       NaN    59   NaN   NaN
3   201708040100  123  3.57632       NaN    59   NaN   NaN
4   201708040120  110  3.12928       NaN    59   NaN   NaN
5   201708040150  100  2.68224       NaN    61   NaN   NaN
6   201708040200  138  4.47040   5.81152    59   NaN   NaN
7   201708040220  120  4.47040       NaN    59   NaN   NaN
8   201708040250  100  4.02336       NaN    59   NaN   NaN
9   201708040300  108  4.02336   5.36448    59   NaN   NaN
10  201708040320   90  3.57632       NaN    59   NaN   NaN
11  201708040350   80  4.02336       NaN    59   NaN   NaN
12  201708040400  102  4.91744   6.70560    58   NaN   NaN
13  201708040420   80  4.47040       NaN    59   NaN   NaN
14  201708040450   80  4.47040       NaN    59   NaN   NaN
15  201708040500  119  5.36448   7.59968    58   NaN   N

Okey, so from here we can actually see that the **`GUST`** column contains information only on an hourly level. That might be useful! Let's keep this in mind.

**REMEMBER**: Whenever starting a data analysis with new dataset, it is highly useful to explore the data by calculating basic statistics from the data.

## Using your own functions in pandas 

Now it's again time to convert temperatures from Fahrenheit to Celsius! Yes, we have already done this many times before, but this time we will learn how to apply self-made functions to data in a pandas DataFrame.
**In short, our task is to define a function for the temperature conversion, and to apply this function for each Celsius value on each row of the DataFrame. Output celsius values should be stored in a new column called** `TEMP_C`.

Knowing how to use your own function in pandas can be really useful when doing your own analyses. Here, we will introduce two different approaches for using function in pandas. First, we will see how we can apply the function row-by-row using a `for`-loop and the `DataFrame.iterrows()`-method, and then we will learn how to apply the method to all rows at once using [DataFrame.apply](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.apply.html).

For both of these approaches, we first need to define our temperature conversion function from Fahrenheit to Celsius:

In [12]:
def fahr_to_celsius(temp_fahrenheit):
    """
    Function to convert Fahrenheit temperature into Celsius.

    Parameters
    ----------

    temp_fahrenheit: int | float
        Input temperature in Fahrenheit (should be a number)
        
    Returns
    -------
    
    Temperature in Celsius (float)
    """

    # Convert the Fahrenheit into Celsius and return it
    converted_temp = (temp_fahrenheit - 32) / 1.8
    return converted_temp

**Note:** with such a simple example, we could use the function direcly on a column in the DataFrame in order to conver the values:

In [13]:
data["TEMP_C"] = fahr_to_celsius(data["TEMP"])
data.head()

,TIME,DIR,SPEED,GUST,TEMP,MAX,MIN,TEMP_C
0,201708040000,114,2.68224,NaN,58,NaN,NaN,14.444444
1,201708040020,100,2.68224,NaN,59,NaN,NaN,15.000000
2,201708040050,100,2.23520,NaN,59,NaN,NaN,15.000000
3,201708040100,123,3.57632,NaN,59,NaN,NaN,15.000000
4,201708040120,110,3.12928,NaN,59,NaN,NaN,15.000000


If we want to do something more complicated, we need to know how to apply the function row-by-row.

### Iterating over rows

We can iterate over the rows of Pandas DataFrame by using the `iterrows()` -method and use the function one row at a time.


When iterating over the rows in our `DataFrame`, it is noteworthy to understand that the Pandas actually keeps track on the `index` value as well. Hence, the contents of a single row actually contains not only the values, but also the `index` of that row (each row is a pandas Series!). 

- Let's see how `iterrows()` works by printing out the `TEMP` value on each row using a `for`-loop:


In [14]:
# Iterate over the rows
for idx, row in data.iterrows():
    # Print the index value
    print('Index:', idx)
    
    # Print the row
    print('Temp F:', row["TEMP"], "\n")
    
    break

Index: 0
Temp F: 58.0 



<div class="alert alert-info">

**break**

When developing a for-loop, you don't always need to go trough the whole loop if you just want to test things out. 
[break](https://www.tutorialspoint.com/python/python_break_statement.htm) statement in Python terminates the current loop after the first iteration and we used it here just to test check out the values on the first row.
    
</div>


We can see that the `idx` variable indeed contains the index value at position 0 (the first row) and the `row` variable contains all the data from that given row stored as a pandas `Series`.

- Let's now create an empty column `TEMP_C` for the Celsius temperatures and update the values into that column using the `fahr_to_celsius` function we defined earlier:


In [15]:
# Create an empty column for the DataFrame where the values will be stored
new_column = "TEMP_C"
data[new_column] = None

# Iterate over the rows 
for idx, row in data.iterrows():
    # Convert the Fahrenheit to Celsius
    celsius = fahr_to_celsius(row['TEMP'])
    
    # Update the value of 'Celsius' column with the converted value
    data.at[idx, new_column] = celsius

<div class="alert alert-info">

**.at or .loc**

Here, you could also use `data.loc[idx, new_column] = celsius` to achieve the same result.
    
</div>

- Let's see what we have now.

In [16]:
data.head(10)

,TIME,DIR,SPEED,GUST,TEMP,MAX,MIN,TEMP_C
0,201708040000,114,2.68224,NaN,58,NaN,NaN,14.4444
1,201708040020,100,2.68224,NaN,59,NaN,NaN,15
2,201708040050,100,2.23520,NaN,59,NaN,NaN,15
3,201708040100,123,3.57632,NaN,59,NaN,NaN,15
4,201708040120,110,3.12928,NaN,59,NaN,NaN,15
5,201708040150,100,2.68224,NaN,61,NaN,NaN,16.1111
6,201708040200,138,4.47040,5.81152,59,NaN,NaN,15
7,201708040220,120,4.47040,NaN,59,NaN,NaN,15
8,201708040250,100,4.02336,NaN,59,NaN,NaN,15
9,201708040300,108,4.02336,5.36448,59,NaN,NaN,15


Great! Now we have converted our temperatures into Celsius by using our self-made function.

### Applying a function

Pandas DataFrames and Series also have a dedicated method `.apply()` for applying functions on columns (or rows!). When using `.apply()`, we pass the function name (without parenthesis!) as an argument:

In [17]:
data["TEMP_C"] = data["TEMP"].apply(fahr_to_celsius)
data.head()

,TIME,DIR,SPEED,GUST,TEMP,MAX,MIN,TEMP_C
0,201708040000,114,2.68224,NaN,58,NaN,NaN,14.444444
1,201708040020,100,2.68224,NaN,59,NaN,NaN,15.000000
2,201708040050,100,2.23520,NaN,59,NaN,NaN,15.000000
3,201708040100,123,3.57632,NaN,59,NaN,NaN,15.000000
4,201708040120,110,3.12928,NaN,59,NaN,NaN,15.000000


**Note:** pay attention which column you are applying the function on! Running this code: `data.apply(fahr_to_celsius)` would not give an error, but the results also don't make much sense.

<div class="alert alert-info">

**Should I use .iterrows() or .apply()?**

We are teaching the `.iterrows()` method because it helps to understand the structure of a DataFrame and the process of looping trough DataFrame rows. However, using `.apply()` is often more efficient in terms of execution time. 
    
    
At this point, the most important thing is that you understand what happens when you are modifying the values in a pandas DataFrame. When doing the course exercises, either of these approaches is ok!
    
</div>

## String manipulation in Pandas

Let's pay attention to columns `TIME`, `GUST`, `SPEED` and `TEMP`:

In [18]:
data.head(20)

,TIME,DIR,SPEED,GUST,TEMP,MAX,MIN,TEMP_C
0,201708040000,114,2.68224,NaN,58,NaN,NaN,14.444444
1,201708040020,100,2.68224,NaN,59,NaN,NaN,15.000000
2,201708040050,100,2.23520,NaN,59,NaN,NaN,15.000000
3,201708040100,123,3.57632,NaN,59,NaN,NaN,15.000000
4,201708040120,110,3.12928,NaN,59,NaN,NaN,15.000000
5,201708040150,100,2.68224,NaN,61,NaN,NaN,16.111111
6,201708040200,138,4.47040,5.81152,59,NaN,NaN,15.000000
7,201708040220,120,4.47040,NaN,59,NaN,NaN,15.000000
8,201708040250,100,4.02336,NaN,59,NaN,NaN,15.000000
9,201708040300,108,4.02336,5.36448,59,NaN,NaN,15.000000


Check also the column data types:

In [19]:
data.dtypes

TIME        int64
DIR         int64
SPEED     float64
GUST      float64
TEMP        int64
MAX       float64
MIN       float64
TEMP_C    float64
dtype: object

- `TIME`column contains several timestamps per hour
- `GUST` seems to be measured only once an hour
- `SPEED` (wind speed), and `TEMP` seem to be measured every 20 minutes (at minutes XX:00, XX:20 and XX:50).

The difference between the time intervals might be a problem as we might not be able to compare e.g. the average wind speeds and the speeds during the gust together as they are measured with different intervals. This kind of mismatch between sampling rates of measurements is actually quite typical when working with real data.

We can solve this issue by **aggregating the wind speeds into hourly data** so that the attributes become comparable.
First we need to be able to group the values by hour. Let's have a look how we can achieve this using the values in the `TIME` column.

Remember, that the original name of the `TIME` column was `'YR--MODAHRMN'`. Based on this information, the first observation in our data set `201708040000` is from 4th of August 2017 at 00:00. There is a systematic pattern in the character string values stored in the `TIME` column, and we can retrieve the information about date+hour by slicing the values in the **`TIME`** -column (i.e. removing the minutes from the end of the value).

Doing this requires two steps:
  1. Convert the `TIME` column from `int` into `str` datatype.
  2. Include only numbers up to hourly accuracy (exclude minutes) by slicing texts

- Let's convert the time into string. And check that the data type changes:

In [20]:
# Convert to string
data['TIME_STR'] = data['TIME'].astype(str)

# Check data types
print("Data type of the column:")
print(data['TIME_STR'].dtypes)

print("\nData type of the first value in column:")
print(type(data.at[0, 'TIME_STR']))

Data type of the column:
object

Data type of the first value in column:
<class 'str'>


Okey it seems that now we indeed have the `TIME` as `str` datatype as well.


<div class="alert alert-info">

**String functions**

**Note:** All the typical `str` functionalities can be applied to Series of text data with syntax `data['mySeries'].str.<functionToUse>()`.
    
</div>


- Now we can extract information about day and hour by including only 10 first characters from the text (i.e. excluding the minute-level information).

In [21]:
# SLice the string
data['TIME_DH'] = data['TIME_STR'].str.slice(start=0, stop=10)

# Let's see what we have
print(data.head())

           TIME  DIR    SPEED  GUST  TEMP  MAX  MIN     TEMP_C      TIME_STR  \
0  201708040000  114  2.68224   NaN    58  NaN  NaN  14.444444  201708040000   
1  201708040020  100  2.68224   NaN    59  NaN  NaN  15.000000  201708040020   
2  201708040050  100  2.23520   NaN    59  NaN  NaN  15.000000  201708040050   
3  201708040100  123  3.57632   NaN    59  NaN  NaN  15.000000  201708040100   
4  201708040120  110  3.12928   NaN    59  NaN  NaN  15.000000  201708040120   

      TIME_DH  
0  2017080400  
1  2017080400  
2  2017080400  
3  2017080401  
4  2017080401  


Nice! Now we have "labeled" the rows based on information about day of the year and hour of the day.

<div class="alert alert-info">

**TASK:**

- Create a new column `TIME_H` which contains the hour of each row as an integer value. For example, the hour for the first row is `0`.
    
</div>

In [22]:
# Slice the string to parse the hour from 'TIME_str' column
data['TIME_H'] = data['TIME_STR'].str.slice(start=8, stop=10)

# Convert the hour text into integer format
data['TIME_H'] = data['TIME_H'].astype(int)

print(data.head())

           TIME  DIR    SPEED  GUST  TEMP  MAX  MIN     TEMP_C      TIME_STR  \
0  201708040000  114  2.68224   NaN    58  NaN  NaN  14.444444  201708040000   
1  201708040020  100  2.68224   NaN    59  NaN  NaN  15.000000  201708040020   
2  201708040050  100  2.23520   NaN    59  NaN  NaN  15.000000  201708040050   
3  201708040100  123  3.57632   NaN    59  NaN  NaN  15.000000  201708040100   
4  201708040120  110  3.12928   NaN    59  NaN  NaN  15.000000  201708040120   

      TIME_DH  TIME_H  
0  2017080400       0  
1  2017080400       0  
2  2017080400       0  
3  2017080401       1  
4  2017080401       1  


## Aggregating data in Pandas by grouping

Next we want to calculate the average temperatures, wind speeds, etc. on an hourly basis to enable us
to compare all of them to each other.

This can be done by aggregating the data, i.e.:

  1. **grouping the data** based on hourly values
  2. Iterating over those groups and calculating the average values of our attributes
  3. Inserting those values into **a new DataFrame** where we store the aggregated data
  
**--> our input data has 3 rows per hour, the aggregated data frame should only have 1 row per hour!**

- Let's first create a new **empty** DataFrame where we will store our aggregated data


In [23]:
# Create a new empty DataFrame
hourly_data = pd.DataFrame()

Let's then **group** our data based on `TIME_DH` attribute that contains the information about the date + hour.

- First, check how many unique values the `TIME_DH` column has using the `.nunique()` method:

In [24]:
print("Number of rows", len(data))
print("Number of unique day + hour combinations:", data["TIME_DH"].nunique())

Number of rows 72
Number of unique day + hour combinations: 24


Indeed, we are currently dealing with data from one day, 3 observations per hour!

- Now, group the data based on `TIME_DH` column:

In [25]:
# Group the data 
grouped = data.groupby('TIME_DH')

- Let's see what we have now.

In [26]:
# What is the type?
print("Type:\n", type(grouped))

# How many?
print("Length:\n", len(grouped))

Type:
 <class 'pandas.core.groupby.groupby.DataFrameGroupBy'>
Length:
 24


Okey, interesting. Now we have a new object with type **`DataFrameGroupBy`**. And it seems that we have 24 individual groups in our data, i.e. **one group for each hour of the day**.

There are several methods we can use for extracting information from the grouped data. See [documentation for Pandas GroupBy objects](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html) for a comprehensive overview. 

**Checking group names:**

In [27]:
# Check the "names" of each group
grouped.groups.keys()

dict_keys(['2017080417', '2017080422', '2017080421', '2017080400', '2017080408', '2017080423', '2017080412', '2017080419', '2017080418', '2017080409', '2017080407', '2017080405', '2017080411', '2017080415', '2017080401', '2017080420', '2017080404', '2017080410', '2017080402', '2017080413', '2017080416', '2017080414', '2017080406', '2017080403'])

**Accessing data for one group:**

As you might have noticed earlier, the first hour in hour data is `2017080400` (midnight at 4th of August in 2017).

- Let's check the contents of the group named `2017080400`. We can get the values of that hour from the grouped object using the `get_group()` -method:

In [28]:
# Specify the time of the first hour (as text)
time1 = '2017080400'

# Select the group
group1 = grouped.get_group(time1)

# Let's see what we have
print(group1)

           TIME  DIR    SPEED  GUST  TEMP  MAX  MIN     TEMP_C      TIME_STR  \
0  201708040000  114  2.68224   NaN    58  NaN  NaN  14.444444  201708040000   
1  201708040020  100  2.68224   NaN    59  NaN  NaN  15.000000  201708040020   
2  201708040050  100  2.23520   NaN    59  NaN  NaN  15.000000  201708040050   

      TIME_DH  TIME_H  
0  2017080400       0  
1  2017080400       0  
2  2017080400       0  


Ahaa! As we can see, a single group contains a **DataFrame** with values only for that specific hour. Let's check the DataType of this group:

In [29]:
type(group1)

pandas.core.frame.DataFrame

So, one group is a pandas DataFrame! This is really useful, because we can now use all the familiar DataFrame methods for calculating statistics etc for this spesific group. 
We can, for example, calculate the average values for all variables using the statistical functions that we have seen already (e.g. mean, std, min, max, median, etc.).

We can do that by using the `mean()` -function that we already used during the Lesson 5. 

- Let's calculate the mean for following attributes all at once:
   - `DIR`, 
   - `SPEED`, 
   - `GUST`, 
   - `TEMP`, 
   - `TEMP_C`

In [30]:
# Specify the columns that will be part of the calculation
mean_cols = ['DIR', 'SPEED', 'GUST', 'TEMP', 'TEMP_C']

# Calculate the mean values all at one go
mean_values = group1[mean_cols].mean()

# Let's see what we have
print(mean_values)

DIR       104.666667
SPEED       2.533227
GUST             NaN
TEMP       58.666667
TEMP_C     14.814815
dtype: float64


Nice, now we have averaged our data and e.g. the mean Celsius temperature seems to be about right when comparing to the original values above. 

The last thing to do is to add these mean values into our DataFrame that we created.
That can be done with `append()` -function in a quite similar manner as with Python lists. In Pandas the data insertion is not done **inplace** (as when appending to Python lists) so we need to specify that we are updating the aggr_data (using the **`=`** sign). We also need to specify that we ignore the index values of our original DataFrame (i.e. the indices of `mean_values`).


In [31]:
# Add the values into our aggr_data DataFrame that we created in the beginning
hourly_data = hourly_data.append(mean_values, ignore_index=True)

# Let's see what we have
print(hourly_data)

          DIR  GUST     SPEED       TEMP     TEMP_C
0  104.666667   NaN  2.533227  58.666667  14.814815


In [32]:
mean_values

DIR       104.666667
SPEED       2.533227
GUST             NaN
TEMP       58.666667
TEMP_C     14.814815
dtype: float64

As we can see, now we have a single row in our new DataFrame where we have aggregated the data based on hourly mean values.
Next we could continue doing and insert the average values from other hours in a similar manner but, of course, that is not
something that we want to do manually (would require repeating these same steps too many times).
Luckily, we can actually iterate over all the groups that we have in our data and do these steps using a **`for`** -loop.

When iterating over the groups in our **`DataFrameGroupBy`** -object
it is important to understand that a single group in our `DataFrameGroupBy` actually contains not only the actual values, but also information about the **`key`** that was used to do the grouping. Hence, when iterating over the data we need to assign the `key` and the values into separate variables.

- Let's see how we can iterate over the groups and print the key and the data from a single group (again using **`break`** to only see what is happening).


In [33]:
# Iterate over groups
for key, group in grouped:
    # Print key and group
    print("Key:\n", key)
    print("\nGroup:\n", group)
    
    # Stop iteration with break command
    break

Key:
 2017080400

Group:
            TIME  DIR    SPEED  GUST  TEMP  MAX  MIN     TEMP_C      TIME_STR  \
0  201708040000  114  2.68224   NaN    58  NaN  NaN  14.444444  201708040000   
1  201708040020  100  2.68224   NaN    59  NaN  NaN  15.000000  201708040020   
2  201708040050  100  2.23520   NaN    59  NaN  NaN  15.000000  201708040050   

      TIME_DH  TIME_H  
0  2017080400       0  
1  2017080400       0  
2  2017080400       0  


Okey so from here we can see that the **`key`** contains the value **`2017080400`** that is the same
as the values in **`TIME_dh`** column. Meaning that we, indeed, grouped the values based on that column.

- Let's see how we can create a DataFrame where we calculate the mean values for all those weather attributes that we were interested in. I will repeate slightly the earlier steps so that you can see and better understand what is happening.


In [34]:
# Create an empty DataFrame for the aggregated values
hourly_data = pd.DataFrame()

# The columns that we want to aggregate
mean_cols = ['DIR', 'SPEED', 'GUST', 'TEMP', 'TEMP_C']

# Iterate over the groups
for key, group in grouped:
   # Aggregate the data
   mean_values = group[mean_cols].mean()

   # Add the ´key´ (i.e. the date+time information) into the aggregated values
   mean_values['TIME_DH'] = key

   # Append the aggregated values into the DataFrame
   hourly_data = hourly_data.append(mean_values, ignore_index=True)

- Let's see what we have now:

In [35]:
print(hourly_data)

           DIR      GUST     SPEED       TEMP     TEMP_C     TIME_DH
0   104.666667       NaN  2.533227  58.666667  14.814815  2017080400
1   111.000000       NaN  3.129280  59.666667  15.370370  2017080401
2   119.333333   5.81152  4.321387  59.000000  15.000000  2017080402
3    92.666667   5.36448  3.874347  59.000000  15.000000  2017080403
4    87.333333   6.70560  4.619413  58.666667  14.814815  2017080404
5   403.000000   7.59968  5.364480  58.666667  14.814815  2017080405
6   110.333333  10.28192  6.854613  58.666667  14.814815  2017080406
7   403.000000   9.83488  6.109547  58.666667  14.814815  2017080407
8   405.000000   6.70560  3.874347  58.666667  14.814815  2017080408
9   695.666667       NaN  3.129280  59.666667  15.370370  2017080409
10  225.000000   5.81152  4.768427  61.666667  16.481481  2017080410
11  241.666667   8.49376  5.513493  64.000000  17.777778  2017080411
12  228.333333   6.70560  5.960533  66.000000  18.888889  2017080412
13  229.666667   8.94080  7.152640

Awesome! Now we have aggregated our data based on daily averages and we have a new DataFrame called `hourly_data` where all those aggregated values are stored.

**Shortcut: Mean for all groups**

We can also achieve the same result by computing the mean of all columns for all groups in the grouped object:

In [36]:
grouped.mean()

,TIME,DIR,SPEED,GUST,TEMP,MAX,MIN,TEMP_C,TIME_H
TIME_DH,,,,,,,,,
2017080400,2.017080e+11,104.666667,2.533227,NaN,58.666667,NaN,NaN,14.814815,0
2017080401,2.017080e+11,111.000000,3.129280,NaN,59.666667,NaN,NaN,15.370370,1
2017080402,2.017080e+11,119.333333,4.321387,5.81152,59.000000,NaN,NaN,15.000000,2
2017080403,2.017080e+11,92.666667,3.874347,5.36448,59.000000,NaN,NaN,15.000000,3
2017080404,2.017080e+11,87.333333,4.619413,6.70560,58.666667,NaN,NaN,14.814815,4
2017080405,2.017080e+11,403.000000,5.364480,7.59968,58.666667,NaN,NaN,14.814815,5
2017080406,2.017080e+11,110.333333,6.854613,10.28192,58.666667,64.0,56.0,14.814815,6
2017080407,2.017080e+11,403.000000,6.109547,9.83488,58.666667,NaN,NaN,14.814815,7
2017080408,2.017080e+11,405.000000,3.874347,6.70560,58.666667,NaN,NaN,14.814815,8


## Extra: datetime

<div class="alert alert-info">

**Pandas datetime**

So far, we have been operating with time using integers and character strings. Python (and pandas) come with more advanced approaches for handling dates and times. In pandas, we can convert dates and times into a new data type [datetime](https://docs.python.org/3.7/library/datetime.html) using [pandas.to_datetime](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html) function. First, it is important to understand the structure of the input data in order to avoid erroneous conversions, and that's why we first focus on string slicing before introducing the datetime functionalities. 
    
Here is one example of how to convert the `TIME_STR`-column in our data set to datetime:
    
```
# Convert to datetime
data["DATE"] = pd.to_datetime(data["TIME_STR"], format='%Y%m%d%H', exact=False)

```

    
In this example, `format` defines the output datetime format according to strftime(format) method: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior. `exact=False` drops out minutes and secods, because they are not included in the specified formatting.
    
</div>




In [37]:
# Convert to datetime
data["DATE"] = pd.to_datetime(data["TIME_STR"], format='%Y%m%d%H', exact=False)
data["DATE"].head()

0   2017-08-04 00:00:00
1   2017-08-04 00:00:00
2   2017-08-04 00:00:00
3   2017-08-04 01:00:00
4   2017-08-04 01:00:00
Name: DATE, dtype: datetime64[ns]

Note: in this case, the data type of the values is `datetime`.

Furthermore, we could group the data based on the new datetime column and retrieve the mean values for each column:


In [38]:
data.groupby("DATE").mean()

,TIME,DIR,SPEED,GUST,TEMP,MAX,MIN,TEMP_C,TIME_H
DATE,,,,,,,,,
2017-08-04 00:00:00,2.017080e+11,104.666667,2.533227,NaN,58.666667,NaN,NaN,14.814815,0
2017-08-04 01:00:00,2.017080e+11,111.000000,3.129280,NaN,59.666667,NaN,NaN,15.370370,1
2017-08-04 02:00:00,2.017080e+11,119.333333,4.321387,5.81152,59.000000,NaN,NaN,15.000000,2
2017-08-04 03:00:00,2.017080e+11,92.666667,3.874347,5.36448,59.000000,NaN,NaN,15.000000,3
2017-08-04 04:00:00,2.017080e+11,87.333333,4.619413,6.70560,58.666667,NaN,NaN,14.814815,4
2017-08-04 05:00:00,2.017080e+11,403.000000,5.364480,7.59968,58.666667,NaN,NaN,14.814815,5
2017-08-04 06:00:00,2.017080e+11,110.333333,6.854613,10.28192,58.666667,64.0,56.0,14.814815,6
2017-08-04 07:00:00,2.017080e+11,403.000000,6.109547,9.83488,58.666667,NaN,NaN,14.814815,7
2017-08-04 08:00:00,2.017080e+11,405.000000,3.874347,6.70560,58.666667,NaN,NaN,14.814815,8


This should be the same result we achieved earlier based on string slicing!

## Finding outliers from the data

Finally, we are ready to do some real data analytics and check whether we are able to find out if there are any outliers in our data suggesting to have a storm (meaning strong winds in this case).

Here, we define an outlier if the **wind speed is 2 times the standard deviation higher than the average wind speed** (column `SPEED`).

- Let's first find out what is the standard deviation and the mean of the Wind speed.


In [39]:
# Calculate standard deviation and average wind speed
std_wind = hourly_data['SPEED'].std()
avg_wind = hourly_data['SPEED'].mean()
print('Std:', std_wind)
print('Mean:', avg_wind)

Std: 1.6405694308360985
Mean: 5.153377777777777


Okey, so the variance in the windspeed tend to be approximately 1.6 meters per second, and the wind speed is approximately 5.2 m/s. 

- Hence, the threshold for a wind speed to be an outlier with our criteria is:


In [40]:
# Calculate the upper threshold for an outlier
upper_threshold = avg_wind + (std_wind*2)
print('Upper threshold for outlier:', upper_threshold)

Upper threshold for outlier: 8.434516639449974


- Let's finally create a column called `OUTLIER` which we update with `True` value, if the windspeed is an outlier, and `False`, if it is not. We do this again by iterating over the rows.


In [41]:
# Create an empty column for outlier info
hourly_data['OUTLIER'] = None

# Iterate over rows
for idx, row in hourly_data.iterrows():
    # Update the 'Outlier' column with True if the wind speed is higher than our threshold value
    if row['SPEED'] > upper_threshold :
        hourly_data.loc[idx, 'OUTLIER'] = True
    else:
        hourly_data.loc[idx, 'OUTLIER'] = False

# Let's see what we have
print(hourly_data)

           DIR      GUST     SPEED       TEMP     TEMP_C     TIME_DH  OUTLIER
0   104.666667       NaN  2.533227  58.666667  14.814815  2017080400    False
1   111.000000       NaN  3.129280  59.666667  15.370370  2017080401    False
2   119.333333   5.81152  4.321387  59.000000  15.000000  2017080402    False
3    92.666667   5.36448  3.874347  59.000000  15.000000  2017080403    False
4    87.333333   6.70560  4.619413  58.666667  14.814815  2017080404    False
5   403.000000   7.59968  5.364480  58.666667  14.814815  2017080405    False
6   110.333333  10.28192  6.854613  58.666667  14.814815  2017080406    False
7   403.000000   9.83488  6.109547  58.666667  14.814815  2017080407    False
8   405.000000   6.70560  3.874347  58.666667  14.814815  2017080408    False
9   695.666667       NaN  3.129280  59.666667  15.370370  2017080409    False
10  225.000000   5.81152  4.768427  61.666667  16.481481  2017080410    False
11  241.666667   8.49376  5.513493  64.000000  17.777778  201708

Okey now we have at least many False values in our `OUTLIER` -column but there seems to be also one True!

- Let's select the rows with potential storm:


In [42]:
# Select rows that were determined as outliers
storm = hourly_data.loc[hourly_data['OUTLIER'] == True]
print(storm)

           DIR      GUST   SPEED       TEMP     TEMP_C     TIME_DH  OUTLIER
14  228.666667  12.96416  8.9408  68.333333  20.185185  2017080414     True


Okey, so indeed, there was one outlier in our data but the wind during that time wasn't that strong as the average speed was only approximately 9 m/s. This is not too strange as we were only looking at data from a single day.

## Repeating the data analysis with larger dataset

Let's continue by executing the steps that we have written this far and use it to explore outlier winds based on whole month of August 2017. So far, we have only processed data for one single day (4th of August).

For this purpose, we change the input file to be **`6591337447542dat_August.txt`** (full record for August 2017). You can find the file in here: [data/6591337447542dat_August.txt](data/6591337447542dat_August.txt).

- Here we will repeat all the steps that we did earlier in one code block so that you can see the full picture:

In [43]:
# Filepath
fp = "data/6591337447542dat_August.txt"

# Read data using varying amount of spaces as separator and specifying * characters as NoData values
data = pd.read_csv(fp, sep='\s+', na_values=['*', '**', '***', '****', '*****', '******'])

# Select only specific columns
select_cols = ['YR--MODAHRMN', 'DIR', 'SPD', 'GUS','TEMP', 'MAX', 'MIN']
data = data[select_cols]

# Rename the columns
name_conversion_dict = {'YR--MODAHRMN': 'TIME', 'SPD': 'SPEED', 'GUS': 'GUST'}
data = data.rename(columns=name_conversion_dict)

# Create column
col_name = 'TEMP_C'
data[col_name] = None

# Iterete over rows and convert tempetarues from Fahrenheits to Celsius
for idx, row in data.iterrows():
    celsius = fahr_to_celsius(row['TEMP'])
    data.loc[idx, col_name] = celsius

# Convert wind speeds from miles to meters per second
data['SPEED'] = data['SPEED']*0.44704
data['GUST'] = data['GUST']*0.44704

# Convert TIME to string and parse date and hour info from the time
data['TIME_STR'] = data['TIME'].astype(str)
data['TIME_DH'] = data['TIME_STR'].str.slice(start=0, stop=10)

# Create empty column for aggregated data
hourly_data = pd.DataFrame()

# Specify the columns which will be used in calculation
mean_cols = ['DIR', 'SPEED', 'GUST', 'TEMP', 'TEMP_C']

# Group the values by hour
grouped = data.groupby('TIME_DH')

# Iterate over groups and update the aggregated DataFrame
for key, group in grouped:
    # Calculate the mean values
    mean_values = group[mean_cols].mean()
    
    # Add the time to the Series
    mean_values['TIME_DH'] = key
    
    # Add the aggregated values into the DataFrame
    hourly_data  = hourly_data .append(mean_values, ignore_index=True)

# Calculate the outlier threshold for the new dataset
std_wind = hourly_data['SPEED'].std()
avg_wind = hourly_data['SPEED'].mean()
upper_threshold = avg_wind + (std_wind*2)

# Detect the outliers
hourly_data['OUTLIER'] = None

for idx, row in hourly_data.iterrows():
    if row['SPEED'] > upper_threshold:
        hourly_data.loc[idx, 'OUTLIER'] = True
    else:
        hourly_data.loc[idx, 'OUTLIER'] = False
        
# Select days with strong winds
storm = hourly_data.loc[hourly_data['OUTLIER'] == True]

After running the analysis with our new dataset, let's explore and see we have.

- Let's start by checking if the average and standard deviation of the windspeed differ from the previous ones

In [44]:
# Windspeed statistics
print('Std:', std_wind)
print('Mean:', avg_wind)

Std: 2.1405899770297245
Mean: 4.1990832704402505


Okey so they are indeed different now! With larger dataset the average wind speed is 4.2 m/s (compared to 5.2 m/s previously). 

- Let's see what we have now in our **`storm`** -variable.

In [45]:
print(storm)

            DIR      GUST      SPEED       TEMP     TEMP_C     TIME_DH  \
10   210.666667  12.51712   9.089813  73.000000  22.777778  2017080110   
11   212.000000  11.62304   8.940800  73.000000  22.777778  2017080111   
12   205.666667  12.51712   9.089813  72.333333  22.407407  2017080112   
86   228.666667  12.96416   8.940800  68.333333  20.185185  2017080414   
104  204.333333  11.17600   8.791787  67.666667  19.814815  2017080508   
132  237.666667  13.85824   9.387840  61.333333  16.296296  2017080612   
230  217.000000  12.51712   8.642773  71.000000  21.666667  2017081014   
280  700.666667  26.82240   8.791787  66.333333  19.074074  2017081216   
301  210.000000       NaN   9.611360  69.000000  20.555556  2017081313   
302  200.000000       NaN   8.493760  67.000000  19.444444  2017081314   
444  195.666667  10.72896   8.493760  71.666667  22.037037  2017081914   
445  204.666667  12.51712   8.940800  69.666667  20.925926  2017081915   
559  328.666667  13.41120   8.493760  

Okey, interesting! Now we can see the the days and hours when it has been stormy in August 2017.
It seems that the storms have usually been during the day time. Let's check if this is the case.

We can easily count how many stormy observations for different hour of the day there has been by
using a [**`value_counts()`**](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.value_counts.html) -function that calculates how many observations per certain value there are in a certain column (works best for categorigal data).

- Let's see the counts for different hours of the day


The average wind speed may not be the perfect measure to find extreme weather conditions. Gust might usually be a better measure for that purpose.

- Let's see what were the strongest gust winds in our dataset by sorting the values using **`sort_values()`** -function.


In [46]:
# Sort values in descending order
gust_sort = storm.sort_values(by='GUST', ascending=False)

# Let's see what we have
print(gust_sort)

            DIR      GUST      SPEED       TEMP     TEMP_C     TIME_DH  \
280  700.666667  26.82240   8.791787  66.333333  19.074074  2017081216   
706  215.666667  14.52880  10.579947  64.333333  17.962963  2017083012   
132  237.666667  13.85824   9.387840  61.333333  16.296296  2017080612   
560  329.333333  13.85824   8.493760  60.666667  15.925926  2017082410   
559  328.666667  13.41120   8.493760  58.666667  14.814815  2017082409   
705  218.333333  13.41120   8.940800  66.000000  18.888889  2017083011   
686  214.000000  13.41120   9.089813  63.000000  17.222222  2017082916   
563  329.666667  13.41120   9.238827  61.333333  16.296296  2017082413   
86   228.666667  12.96416   8.940800  68.333333  20.185185  2017080414   
10   210.666667  12.51712   9.089813  73.000000  22.777778  2017080110   
445  204.666667  12.51712   8.940800  69.666667  20.925926  2017081915   
230  217.000000  12.51712   8.642773  71.000000  21.666667  2017081014   
12   205.666667  12.51712   9.089813  

There! There was one hour with quite extraordinary gust wind in our data happening at 12th of August in 2017 - the day when [Kiira hit Helsinki](https://yle.fi/uutiset/osasto/news/saturday_night_storm_downs_trees_cuts_electricity_in_the_south/9773250).


